# 1.Build a NLP Language model for text generation involves train a neural network to predict the next word in a sequence of words.

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Sample text data
text = "NLP stands for Natural Language Processing."

# Preprocess the text
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences and labels
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
x, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x, y, epochs=100, verbose=1)

# Function to generate text
def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generate text
generated_text = generate_text("Breaking down text into individual words or tokens.", 20, model, max_sequence_length)
print(generated_text)


Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.9443 - accuracy: 0.2000
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: 1.9328 - accuracy: 0.4000
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 1.9214 - accuracy: 0.4000
Epoch 4/100
1/1 [==============================] - 0s 7ms/step - loss: 1.9097 - accuracy: 0.4000
Epoch 5/100
1/1 [==============================] - 0s 13ms/step - loss: 1.8977 - accuracy: 0.4000
Epoch 6/100
1/1 [==============================] - 0s 14ms/step - loss: 1.8852 - accuracy: 0.4000
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 1.8721 - accuracy: 0.4000
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 1.8582 - accuracy: 0.4000
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 1.8433 - accuracy: 0.4000
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 1.8273 - accuracy: 0.4000
Epoch 11/100
1/1 [======

# 2.Build a Speech to Text model.

In [22]:
import speech_recognition as sr

def speech_to_text():
    # Initialize the recognizer
    recognizer = sr.Recognizer()

    # Capture audio from the microphone
    with sr.Microphone() as source:
        print("Say something:Book")
        audio = recognizer.listen(source)

    try:
        # Use Google Speech Recognition to convert audio to text
        text = recognizer.recognize_google(audio)
        print("You said: {}".format(text))
    except sr.UnknownValueError:
        print("Could not understand audio")
    except sr.RequestError as e:
        print("Error with the speech recognition service; {0}".format(e))

# Call the function
speech_to_text()


Say something:Book
You said: han FIR papa papa


# 3.Build a Text to Speech model.

In [17]:
from gtts import gTTS
import os

def text_to_speech(text, language='en', filename='output.mp3'):
    # Create a gTTS object
    tts = gTTS(text=text, lang=language, slow=False)

    # Save the audio file
    tts.save(filename)

    # Play the generated audio file
    os.system(f'start {filename}')

# Example usage
text = "Hello, this is a Text-to-Speech model built with gTTS."
text_to_speech(text)


# 4.Build a NLP Language model to detect the sentence/word error in the text corpus.

In [20]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

def detect_errors(sentence):
    # Load pre-trained BERT model and tokenizer for sequence classification
    model_name = 'bert-base-uncased'
    model = BertForSequenceClassification.from_pretrained(model_name)
    tokenizer = BertTokenizer.from_pretrained(model_name)

    # Tokenize input sentence
    inputs = tokenizer(sentence, return_tensors="pt")

    # Get model prediction
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_label = torch.argmax(probabilities, dim=1).item()

    # For simplicity, let's assume label 1 indicates an error
    return predicted_label == 1

# Example usage
text = "This is an example sentece with sme mistkes."
is_error = detect_errors(text)
print("Is there an error in the text?", is_error)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Is there an error in the text? False


# 5.Build a Language model to correct the error in the text

In [19]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def correct_errors(input_text):
    # Load pre-trained GPT-2 model and tokenizer
    model_name = 'gpt2'
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate corrected text
    output = model.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    corrected_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return corrected_text

# Example usage
input_text = "This is an example sentece with sme mistkes."
corrected_text = correct_errors(input_text)
print("Original Text:", input_text)
print("Corrected Text:", corrected_text)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

C:\Users\TmC\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\TmC\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original Text: This is an example sentece with sme mistkes.
Corrected Text: This is an example sentece with sme mistkes.

If you have any questions, please feel free to contact me.
